In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

Load libraries

In [ ]:
from fastai.imports import *
from fastai.structured import *
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn import metrics
from dill import load_session

Load environment

In [1]:
dill.load_session('tmp/globalsave.pkl')

NameError: name 'dill' is not defined

## Future work plan
 __Goal: Integrate the time information into the analysis.__
        - Seasonality analysis: important to select the correct test set (few daya at the beginning and more at the end)        
        -Info about geo location of the shop, to calculate distance from surroundings location.

## Seasonality

In [ ]:
def all_POSs_aggregate(x):
    d = {}
    d['Dayofweek'] = x['Dayofweek'].iloc[0]
    d['n_shops'] = x['store_code'].nunique()
    d['n_open_shops'] = np.count_nonzero(x['volume_day'])
    d['tot_volume'] = x['volume_day'].sum()
    return pd.Series(d, index=['Dayofweek','n_shops','n_open_shops','tot_volume'])

seasonality = df_aggregate_per_day_short_with_info_filter.groupby('date').apply(all_POSs_aggregate)
seasonality['norm_volume'] = seasonality.tot_volume/seasonality.n_shops
seasonality['norm_volume_open'] = seasonality.tot_volume/seasonality.n_open_shops
seasonality.head()

In [ ]:
plt.plot(seasonality.norm_volume);
plt.axvline(x='2016-11-10');
plt.axvline(x='2016-04-01');
plt.xlabel('Date');
plt.ylabel('TotVolume/#POSs');

We see big variability at the beginning of the time series, suggesting a seasonality effect (ups and downs). 
The down picks are due to the sundays.

In [ ]:
plt.plot(seasonality.norm_volume_open);
plt.axvline(x='2016-11-10');
plt.axvline(x='2016-04-01');
plt.xlabel('Date');
plt.ylabel('TotVolume/#POSs open');

In this plot, we normalized the volume by the shops that sold something, since we don't know if no sell is shop closed or shop not selling.

In [ ]:
plt.plot(seasonality.n_shops, marker = 'o')
plt.axvline(x='2016-11-10');
plt.axvline(x='2016-04-01');
plt.xlabel('Date');
plt.ylabel('#POSs');

When we control for the number of shops for each time point, we clearly see that the big variability observed at the beginning is due to few observation available. It seems that more and more information were available along time. It seems that the information about sell started to be increased by the end of 2016 (vertical line on the 10th November 2016). Already at April 2016 we see an increase in the number of shops. The last drop we see at the end of the time series might be due to just lag in receiving information.